In [1]:
#Import Dependencies
import numpy as np
from citipy import citipy
import requests
import time
from datetime import datetime
import pandas as pd

In [2]:
## 3.Create a new set of 2,000 random latitudes and longitudes

In [3]:
#Create the set of 2,000 random longitudes and latitudes
lats = np.random.uniform(-90.000, 90.000, size= 2000)
lngs = np.random.uniform(-180.000, 180.000, size = 2000)

#pack the lats and longs together by zipping them with the zip() function
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
#Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [5]:
## 4.Get the nearest city using the citipy module.

In [6]:
#Start with a list to hold the cities
cities = []

#identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #if the city is unique, add it to the cities list
    if city not in cities:
        cities.append(city)
#print the city count to confirm a sufficent list
len(cities)

754

In [14]:
## 5.Perform an API call with the OpenWeatherMap.

In [15]:
#import the API key
from config import weather_api_key

In [16]:
#Starting URL for Weather API Call from https://openweathermap.org/current#name
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [17]:
# Practicing with Boston Create an endpoint URL for a city.
city_url = url + "&q=" + "Boston"

In [20]:
# Make a 'Get' request for the city weather.
city_weather = requests.get(city_url)
city_weather
#create output for request errors in case the url doesnt work and doesnt produce a status of 200
if city_weather.status_code == 200:
    print(f"City Weather found.")
else:
    print(f"City weather not found.")


City Weather found.


In [21]:
#get the text of the 'get'request
city_weather.text

'{"coord":{"lon":-71.0598,"lat":42.3584},"weather":[{"id":804,"main":"Clouds","description":"overcast clouds","icon":"04d"}],"base":"stations","main":{"temp":46.6,"feels_like":39.72,"temp_min":45,"temp_max":48.2,"pressure":1009,"humidity":81},"visibility":10000,"wind":{"speed":17.27,"deg":40},"clouds":{"all":90},"dt":1618159290,"sys":{"type":1,"id":3486,"country":"US","sunrise":1618135751,"sunset":1618183246},"timezone":-14400,"id":4930956,"name":"Boston","cod":200}'

In [27]:
city_weather = requests.get(city_url).json()
city_lat = city_weather["coord"]["lat"]
city_lng = city_weather["coord"]["lon"]
city_max_temp = city_weather["main"]["temp_max"]
city_humidity = city_weather["main"]["humidity"]
city_clouds = city_weather["clouds"]["all"]
city_wind = city_weather["wind"]["speed"]
city_country = city_weather["sys"]["country"]
city_description = city_weather["weather"][0]["description"]
print(city_lat, city_lng, city_max_temp, city_humidity, city_clouds, city_description)

42.3584 -71.0598 48.2 81 90 overcast clouds


In [28]:
# Make a 'Get' request for the city weather.
city_weather = requests.get(city_url)
city_weather
#create output for request errors in case the url doesnt work and doesnt produce a status of 200
if city_weather.status_code == 200:
    print(f"City Weather found.")
else:
    print(f"City weather not found.")

City Weather found.


In [29]:
#get the text of the 'get'request
city_weather.json()

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 46.6,
  'feels_like': 39.72,
  'temp_min': 45,
  'temp_max': 48.2,
  'pressure': 1009,
  'humidity': 81},
 'visibility': 10000,
 'wind': {'speed': 17.27, 'deg': 40},
 'clouds': {'all': 90},
 'dt': 1618159290,
 'sys': {'type': 1,
  'id': 3486,
  'country': 'US',
  'sunrise': 1618135751,
  'sunset': 1618183246},
 'timezone': -14400,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [30]:
## 6.Retrieve the following information from the API call:
#- Latitude and longitude
#- Maximum temperature
#- Percent humidity
#- Percent cloudiness
#- Wind speed
#- Weather description (for example, clouds, fog, light rain, clear sky)

In [31]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters that start at 1 not 0.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [32]:
#use enumerate() to retrieve both the citie index and city name in the same for loop
#My API key was temporarily blocked after exceeding the request limit per minute to 
#I added some code to have the requests sleep for 60 seconds just before reaching the limit

API_REQ_MAX_PER_SEC = 60 #API request limit per second

for i, city in enumerate(cities):
    #don't exceed API rate limit
    if i % (API_REQ_MAX_PER_SEC - 1) == 0 and i != 0:
        print("Waiting for next request window...")
        time.sleep(60)

    #group the cities in dets of 50 for logging purposes
    if (i % 50 == 0 and i > 50):
        set_count +=1
        record_count = 1
    #creat and endpoint url with each city
    city_url = url + "&q=" + cities[i]  
    
    #log the url, record and set numbers and the city name
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    #add 1 to the record count
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_descript = city_weather["weather"][0]["description"]
        #city_weather_descript = []   
        #for entry in city_weather["weather"]:
        #     city_weather_descript.append(entry["description"])
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Cloudiness": city_clouds,
                          "Country": city_country,
                          "Current Description": city_weather_descript,
                          "Date": city_date,  
                          "Humidity": city_humidity,                          
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Wind Speed": city_wind})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | mahebourg
Processing Record 2 of Set 1 | khurda
Processing Record 3 of Set 1 | kushmurun
Processing Record 4 of Set 1 | yatou
Processing Record 5 of Set 1 | hobyo
Processing Record 6 of Set 1 | puerto baquerizo moreno
Processing Record 7 of Set 1 | port blair
Processing Record 8 of Set 1 | bolungarvik
City not found. Skipping...
Processing Record 9 of Set 1 | ribeira grande
Processing Record 10 of Set 1 | taihe
Processing Record 11 of Set 1 | thompson
Processing Record 12 of Set 1 | aswan
Processing Record 13 of Set 1 | new norfolk
Processing Record 14 of Set 1 | shache
Processing Record 15 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 16 of Set 1 | ostrovnoy
Processing Record 17 of Set 1 | saint-philippe
Processing Record 18 of Set 1 | albany
Processing Record 19 of Set 1 | novyy urengoy
Processing Record 20 of Set 1 | bluff
Processing Record 21 of Set 1 | lavrentiya
Processing Record 22 of Set 1 | busselton
Processing Record 

Processing Record 33 of Set 3 | tumannyy
City not found. Skipping...
Processing Record 34 of Set 3 | bolshiye uki
City not found. Skipping...
Processing Record 35 of Set 3 | salalah
Processing Record 36 of Set 3 | sept-iles
Processing Record 37 of Set 3 | podyuga
Processing Record 38 of Set 3 | korampallam
Processing Record 39 of Set 3 | weligama
Processing Record 40 of Set 3 | vestmannaeyjar
Processing Record 41 of Set 3 | avarua
Processing Record 42 of Set 3 | mabaruma
Processing Record 43 of Set 3 | odweyne
Processing Record 44 of Set 3 | lalmohan
Processing Record 45 of Set 3 | trat
Processing Record 46 of Set 3 | enugu
Processing Record 47 of Set 3 | sur
Processing Record 48 of Set 3 | shingu
Processing Record 49 of Set 3 | sao joao da barra
Processing Record 50 of Set 3 | dikson
Processing Record 1 of Set 4 | ahipara
Processing Record 2 of Set 4 | gushikawa
Processing Record 3 of Set 4 | broome
Processing Record 4 of Set 4 | codrington
Processing Record 5 of Set 4 | bilibino
Proc

Processing Record 24 of Set 7 | tual
Processing Record 25 of Set 7 | irbeyskoye
Processing Record 26 of Set 7 | ust-tsilma
Processing Record 27 of Set 7 | san cristobal
Processing Record 28 of Set 7 | revda
Processing Record 29 of Set 7 | berlevag
Processing Record 30 of Set 7 | virginia beach
Processing Record 31 of Set 7 | mbanza-ngungu
Processing Record 32 of Set 7 | tocopilla
Processing Record 33 of Set 7 | chadiza
Processing Record 34 of Set 7 | krasnoselkup
Processing Record 35 of Set 7 | borgarnes
Processing Record 36 of Set 7 | sivaki
Processing Record 37 of Set 7 | banikoara
Processing Record 38 of Set 7 | sambava
Processing Record 39 of Set 7 | ankang
Processing Record 40 of Set 7 | lexington
Processing Record 41 of Set 7 | port-gentil
Processing Record 42 of Set 7 | saint-augustin
Processing Record 43 of Set 7 | port hedland
Processing Record 44 of Set 7 | ardakan
Processing Record 45 of Set 7 | severomorsk
Processing Record 46 of Set 7 | masumbwe
Processing Record 47 of Set

Processing Record 12 of Set 11 | seminole
Processing Record 13 of Set 11 | sovetskiy
Processing Record 14 of Set 11 | darhan
Processing Record 15 of Set 11 | parfenyevo
Processing Record 16 of Set 11 | ngukurr
City not found. Skipping...
Processing Record 17 of Set 11 | sitka
Processing Record 18 of Set 11 | suhut
Processing Record 19 of Set 11 | tshikapa
Processing Record 20 of Set 11 | teruel
Processing Record 21 of Set 11 | nkhotakota
Processing Record 22 of Set 11 | chernaya kholunitsa
Processing Record 23 of Set 11 | tessalit
Processing Record 24 of Set 11 | bubaque
Processing Record 25 of Set 11 | vardo
Processing Record 26 of Set 11 | kalmunai
Processing Record 27 of Set 11 | ostroda
Processing Record 28 of Set 11 | kalakamati
Processing Record 29 of Set 11 | saint-joseph
Processing Record 30 of Set 11 | anar darreh
City not found. Skipping...
Processing Record 31 of Set 11 | san martin
Processing Record 32 of Set 11 | vila
Processing Record 33 of Set 11 | marzuq
Processing Reco

Processing Record 46 of Set 14 | opunake
Processing Record 47 of Set 14 | solnechnyy
Processing Record 48 of Set 14 | ashqelon
Processing Record 49 of Set 14 | adrar
Processing Record 50 of Set 14 | kaliua
Processing Record 1 of Set 15 | jacqueville
Processing Record 2 of Set 15 | oranjestad
Processing Record 3 of Set 15 | brigantine
Processing Record 4 of Set 15 | mettet
-----------------------------
Data Retrieval Complete      
-----------------------------


In [36]:
len(city_data)

695

In [37]:
## 7.Add the data to a new DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Cloudiness,Country,Current Description,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mahebourg,40,MU,light rain,2021-04-11 16:46:21,78,-20.4081,57.7000,80.60,6.91
1,Khurda,20,IN,haze,2021-04-11 16:47:26,83,20.1833,85.6167,82.40,6.91
2,Kushmurun,92,KZ,overcast clouds,2021-04-11 16:47:27,94,52.4503,64.6311,29.77,4.00
3,Yatou,100,CN,light rain,2021-04-11 16:47:27,84,37.1500,122.3833,49.50,8.30
4,Hobyo,94,SO,overcast clouds,2021-04-11 16:38:29,64,5.3505,48.5268,82.96,10.58
5,Puerto Baquerizo Moreno,75,EC,broken clouds,2021-04-11 16:47:27,83,-0.9000,-89.6000,78.80,4.61
6,Port Blair,99,IN,overcast clouds,2021-04-11 16:47:28,75,11.6667,92.7500,82.44,4.27
7,Ribeira Grande,75,PT,broken clouds,2021-04-11 16:47:28,100,38.5167,-28.7000,62.60,3.44
8,Taihe,100,CN,overcast clouds,2021-04-11 16:47:28,92,30.8712,105.3845,54.90,2.98
9,Thompson,90,CA,overcast clouds,2021-04-11 16:47:29,79,55.7435,-97.8558,21.20,10.36


In [38]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]

#city_data_df.dtypes
#lat and lng come up as objects, convert them to string so they can be mapped

city_data_df["Lat"] = city_data_df["Lat"].astype("float64").map("{:.2f}".format)
city_data_df["Lng"] = city_data_df["Lng"].astype("float64").map("{:.2f}".format)



city_data_df.head(10)


 

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Mahebourg,MU,-20.41,57.70,80.60,78,40,6.91,light rain
1,Khurda,IN,20.18,85.62,82.40,83,20,6.91,haze
2,Kushmurun,KZ,52.45,64.63,29.77,94,92,4.00,overcast clouds
3,Yatou,CN,37.15,122.38,49.50,84,100,8.30,light rain
4,Hobyo,SO,5.35,48.53,82.96,64,94,10.58,overcast clouds
5,Puerto Baquerizo Moreno,EC,-0.90,-89.60,78.80,83,75,4.61,broken clouds
6,Port Blair,IN,11.67,92.75,82.44,75,99,4.27,overcast clouds
7,Ribeira Grande,PT,38.52,-28.70,62.60,100,75,3.44,broken clouds
8,Taihe,CN,30.87,105.38,54.90,92,100,2.98,overcast clouds
9,Thompson,CA,55.74,-97.86,21.20,79,90,10.36,overcast clouds


In [42]:
## 8.Export the DataFrame as a CSV file, and save it as WeatherPy_Database.csv in the Weather_Database folder
output_data_file = "WeatherPy_Database.csv"
city_data_df.to_csv(output_data_file, index_label = "City_ID")